In [43]:
import pandas as pd

# Load the dataset
df = pd.read_csv('shopping_trends.csv')

def lower_case_column_names(columns):
    df.columns = [i.lower() for i in df.columns]
    return df

lower_case_column_names(df)

df.columns = df.columns.str.replace(' ', '_')

df.head()


,customer_id,age,gender,item_purchased,category,purchase_amount_(usd),location,size,color,season,review_rating,subscription_status,payment_method,shipping_type,discount_applied,promo_code_used,previous_purchases,preferred_payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               3900 non-null   int64  
 1   age                       3900 non-null   int64  
 2   gender                    3900 non-null   object 
 3   item_purchased            3900 non-null   object 
 4   category                  3900 non-null   object 
 5   purchase_amount_(usd)     3900 non-null   int64  
 6   location                  3900 non-null   object 
 7   size                      3900 non-null   object 
 8   color                     3900 non-null   object 
 9   season                    3900 non-null   object 
 10  review_rating             3900 non-null   float64
 11  subscription_status       3900 non-null   object 
 12  payment_method            3900 non-null   object 
 13  shipping_type             3900 non-null   object 
 14  discount

In [30]:
number_of_products_purchased = df.groupby('customer_id')['item_purchased'].nunique().reset_index()

total_purchase_amount = df.groupby('customer_id')['purchase_amount_(usd)'].sum().reset_index()

average_purchase_amount = df.groupby('customer_id')['purchase_amount_(usd)'].mean().reset_index()

final_data = df[['customer_id', 'payment_method']]
features = [total_purchase_amount, average_purchase_amount, number_of_products_purchased, df[['customer_id', 'age', 'gender', 'frequency_of_purchases_numerical']]]
for feature in features:
    final_data = pd.merge(final_data, feature, on='customer_id')


customer_demographics = final_data[['age', 'gender']]

In [41]:
# Ensure the data is one-hot encoded for products (each product is a column)
data = pd.get_dummies(df, columns=['item_purchased'], prefix='', prefix_sep='')

# Group data by customer and aggregate product purchases
basket = df.groupby('customer_id').sum()


C:\Users\Serhat\AppData\Local\Temp\ipykernel_4888\14366167.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  basket = df.groupby('customer_id').sum()


In [45]:
df.item_purchased.unique()

array(['Blouse', 'Sweater', 'Jeans', 'Sandals', 'Sneakers', 'Shirt',
       'Shorts', 'Coat', 'Handbag', 'Shoes', 'Dress', 'Skirt',
       'Sunglasses', 'Pants', 'Jacket', 'Hoodie', 'Jewelry', 'T-shirt',
       'Scarf', 'Hat', 'Socks', 'Backpack', 'Belt', 'Boots', 'Gloves'],
      dtype=object)

In [46]:
df.customer_id.unique()

array([   1,    2,    3, ..., 3898, 3899, 3900], dtype=int64)

In [51]:
import pandas as pd
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Ensure the data is one-hot encoded for products (each product is a column)
data_encoded = pd.get_dummies(df, columns=['item_purchased'], prefix='', prefix_sep='')

# Group data by customer and aggregate product purchases
basket = data_encoded.groupby('customer_id').sum()

# Apply the Apriori algorithm to find frequent itemsets
min_support = 0.01  # Adjust as needed
frequent_itemsets = apriori(basket, min_support=min_support, use_colnames=True)

# Generate association rules with a minimum confidence threshold
min_confidence = 0.5  # Adjust as needed
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=min_confidence)

# Sort the rules by lift (or another metric) to find the most interesting associations
sorted_rules = rules.sort_values(by=['lift'], ascending=False)

# Print the resulting association rules
print(sorted_rules)


C:\Users\Serhat\AppData\Local\Temp\ipykernel_4888\3769435972.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  basket = data_encoded.groupby('customer_id').sum()
C:\Users\Serhat\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 55.0

In [54]:
# Check unique values in the 'product' column
unique_products = df['item_purchased'].unique()
print(unique_products)

# Check data types in the DataFrame
print(data.dtypes)


['Blouse' 'Sweater' 'Jeans' 'Sandals' 'Sneakers' 'Shirt' 'Shorts' 'Coat'
 'Handbag' 'Shoes' 'Dress' 'Skirt' 'Sunglasses' 'Pants' 'Jacket' 'Hoodie'
 'Jewelry' 'T-shirt' 'Scarf' 'Hat' 'Socks' 'Backpack' 'Belt' 'Boots'
 'Gloves']
customer_id                           int64
age                                   int64
gender                                int32
category                              int32
purchase_amount_(usd)                 int64
location                              int32
size                                  int32
color                                 int32
season                                int32
review_rating                       float64
subscription_status                   int32
payment_method                       object
shipping_type                        object
discount_applied                      int32
promo_code_used                       int32
previous_purchases                    int64
preferred_payment_method              int32
frequency_of_purchases   